## sprint0: global

In [31]:
#!pip install -q keras-nightly

In [32]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
# import before
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.15.0
2.15.0


In [33]:
import numpy as np
print(np.__version__)
# setting random_state
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

1.23.5


### libraries

In [34]:
# libraries
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl, sklearn


In [35]:
# fix random_state
def fixRandomState(fixed_state: int = RANDOM_STATE):
  np.random.seed(fixed_state)
  tf.random.set_seed(fixed_state)

# exception
def exception(requirement: bool, content):
  if(requirement == False): raise ValueError(content)
def catchException(ex: Exception):
  print(type(ex), ex.args) 
  exception(False, ex)

# message
def mesVerbose(flag: bool, verbose, func_dir: str=""):
  if(flag == False): return
  print("__verbose__:", func_dir, verbose)
def mesWarning(note, func_dir: str=""):
  print("__warning__:", func_dir, str(note) + "!")

# dynamic value config
dynamic_value_config = defaultdict(lambda x: None)
def encode_dvc(name: str) -> str:
  name = name.lower()
  return name.replace(" ","_")
def add_dvc(name: str, value):
  dynamic_value_config[encode_dvc(name)] = value
def get_dvc(name: str):
  value = dynamic_value_config[encode_dvc(name)]
  exception(value != None, 'dvc is none')
  return value
def show_dvc():
  mesVerbose(True, "", " > show_dvc:")
  for key, value in dynamic_value_config.items():
    print(f'{key}: {value}')

### view

In [36]:
class View:
  # Singleton Pattern
  _instance = None
  def __new__(cls):
    if cls._instance is None: cls._instance = super(View, cls).__new__(cls)
    return cls._instance

  def over(self, val) -> tuple:
    try:
      mesVerbose(True, (type(val), val.shape), "View > over:")
    except:
      mesVerbose(True, (type(val), "no shape"), "View > over:")

  def plotModel(self, model, show_name: bool=False):
    return keras.utils.plot_model(model,
      show_layer_names=show_name, show_layer_activations=True,
      show_shapes=True, show_dtype=True)

  def debugModel(self, model):
    model.summary(show_trainable=True)

view = View()

## sprint1: using available model from keras

### const value 

In [37]:
# const data
IMAGE_TARGET_SIZE = (64, 64)
IMAGE_COLOR_MODE = 'rgb'
INPUT_SHAPE = (64, 64, 3)
CLASS_MODE = 'binary'
OUTPUT_SHAPE = (1)
BATCH_SIZE = 64

In [38]:
DIR_TRAIN = '/content/Ipynb_Pipeline_Guide/Train'
DIR_VALID = '/content/Ipynb_Pipeline_Guide/Validation'

In [39]:
# const checkpoint
NAME_MODEL_INIT = 'model_after_init_state.keras'
NAME_MODEL_EPOCH = 'model_after_epoch_{epoch}.keras'
NAME_MODEL_TEST = 'model_after_test.keras'

### dataflow

In [40]:
#%rm -rf Ipynb_Pipeline_Guide_2

In [41]:
!git clone -b master https://github.com/QuanHoangNgoc/Ipynb_Pipeline_Guide

fatal: destination path 'Ipynb_Pipeline_Guide' already exists and is not an empty directory.


### ImageLoading

In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class ImageLoading:
  def __init__(self, batch_size: int, class_mode: str,
              target_size: tuple, color_mode='rgb',
              seed: int = RANDOM_STATE):
    self.batch_size, self.class_mode = batch_size, class_mode
    self.target_size, self.color_mode = target_size, color_mode
    self.seed = seed

  def convertToTFDataset(self, data_generator):
    s, s2 = INPUT_SHAPE, OUTPUT_SHAPE
    dataset = tf.data.Dataset.from_generator(
      lambda: data_generator,
      output_signature=(
          tf.TensorSpec(shape=(None, s[0], s[1], s[2]), dtype=tf.float32),
          tf.TensorSpec(shape=(None, ), dtype=tf.float32)
      )
    )
    return dataset

  def getGenerator(self, split: float, rng: bool):
    gen = ImageDataGenerator(validation_split=split, rescale = 1./255)
    if(rng):
      gen = ImageDataGenerator(validation_split=split, rescale = 1./255,
        rotation_range=25, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True, fill_mode='nearest')
    return gen

  def flowBatches(self, dir: str, split: float=0.999, rng: bool=False):
    gen = self.getGenerator(split, rng)
    data_gen = gen.flow_from_directory(dir, subset='validation',
      batch_size=self.batch_size, class_mode=self.class_mode, target_size=self.target_size, color_mode=self.color_mode,
      seed=self.seed)
    view.over(data_gen)
    return data_gen

In [43]:
loading = ImageLoading(BATCH_SIZE, CLASS_MODE, IMAGE_TARGET_SIZE, IMAGE_COLOR_MODE)

In [44]:
loading.flowBatches(DIR_TRAIN, 0.05, True)

Found 7999 images belonging to 2 classes.
__verbose__: View > over: (<class 'keras.src.preprocessing.image.DirectoryIterator'>, 'no shape')


In [45]:
loading.flowBatches(DIR_VALID, 0.05, False)

Found 1129 images belonging to 2 classes.
__verbose__: View > over: (<class 'keras.src.preprocessing.image.DirectoryIterator'>, 'no shape')


### base model

In [46]:
# base model VGG16
base_model = keras.applications.VGG16(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')

In [47]:
# freeze base model
add_dvc('number_base_unfreeze', -5)
for layer in base_model.layers[:get_dvc('number base unfreeze')]:
  layer.trainable = False

In [48]:
show_dvc()

__verbose__:  > show_dvc: 
number_base_unfreeze: -5


### main model

In [49]:
# main model
from keras import Sequential
from keras import layers
from keras.layers import Conv2D, BatchNormalization, Flatten, Dense, MaxPooling2D

# Building Model
model=Sequential()
model.add(base_model.input)
model.add(base_model)
model.add(layers.Dropout(.2))
model.add(Conv2D(512, (3, 3), strides=(1,1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(layers.Dropout(.1))
model.add(Conv2D(128, (3, 3), strides=(1,1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(layers.Dropout(.1))
model.add(Conv2D(384, (3, 3), strides=(1,1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(layers.Dropout(.1))
model.add(Conv2D(384, (3, 3), strides=(1,1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(layers.Dropout(.1))
model.add(Conv2D(500, (3, 3), strides=(1,1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, strides=(2,2), padding='same'))
# Add new layers
FC = 2048
E = 1
model.add(Flatten())
model.add(Dense(FC , activation='relu'))
model.add(layers.Dropout(.2))
model.add(Dense(FC, activation='relu'))
model.add(layers.Dropout(.2))
model.add(Dense(FC, activation='relu'))
model.add(layers.Dropout(.2))
model.add(Dense(E, activation='sigmoid'))
# debug model
# view.debugModel(model)
# view.plotModel(model)

### ModelSaving

In [50]:
from keras.callbacks import ModelCheckpoint

class ModelSaving:
  def __init__(self, name_init: str, name_epoch: str, name_test: str):
    self.name_init, self.name_epoch, self.name_test = name_init, name_epoch, name_test
    self.mcp = ModelCheckpoint(name_epoch)

  def getModelCheckpoint(self):
    return self.mcp

  def getModelFromInitSatte(self):
    return keras.saving.load_model(self.name_init)
  def getModelFromEpoch(self, id: int = -1):
    if(id == -1):
      return keras.saving.load_model(self.name_epoch)
    return keras.saving.load_model(self.name_epoch.format(epoch = id))

  def saveInitState(self, model):
    return model.save(self.name_init)
  def saveTestedModel(self, model):
    return model.save(self.name_test)

saving = ModelSaving(NAME_MODEL_INIT, NAME_MODEL_EPOCH, NAME_MODEL_TEST)

### ModelPrint

In [51]:
from keras.callbacks import Callback

# customize engineering Callback
class ModelPrint(Callback):
  def __init__(self, big_validation, epoch_per_print: int=10):
    super().__init__()
    self.data_out_sample =  big_validation
    self.epp = epoch_per_print

  def on_epoch_end(self, epoch, logs=None):
    if not hasattr(self, 'model'):
      raise ValueError("callback can not access into model")

    if(epoch % self.epp == 0):
      mesVerbose(True, self.model.evaluate(self.data_out_sample), "ModelPrint > on_epoch_end:")

### compile

In [52]:
# compile
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

LR_ALPHA = 0.001
model.compile(optimizer=Adam(LR_ALPHA), loss='binary_crossentropy', metrics=['accuracy'])
view.debugModel(model)

Model: "sequential_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 vgg16 (Functional)          (None, 2, 2, 512)         1471468   Y          
                                                       8                    
                                                                            
 dropout_8 (Dropout)         (None, 2, 2, 512)         0         Y          
                                                                            
 conv2d_5 (Conv2D)           (None, 2, 2, 512)         2359808   Y          
                                                                            
 batch_normalization_5 (Bat  (None, 2, 2, 512)         2048      Y          
 chNormalization)                                                           
                                                                            
 dropout_9 (Dropout)         (None, 2, 2, 512)        

In [53]:
saving.saveInitState(model)

## sprint2: fitting and testing

In [54]:
add_dvc('split_flow_from_dir', 0.05)

In [55]:
train_data_gen = loading.flowBatches(DIR_TRAIN, get_dvc('split_flow_from_dir'), True)

Found 7999 images belonging to 2 classes.
__verbose__: View > over: (<class 'keras.src.preprocessing.image.DirectoryIterator'>, 'no shape')


In [56]:
valid_data_gen = loading.flowBatches(DIR_VALID, get_dvc('split_flow_from_dir'), False)

Found 1129 images belonging to 2 classes.
__verbose__: View > over: (<class 'keras.src.preprocessing.image.DirectoryIterator'>, 'no shape')


In [57]:
big = loading.flowBatches(DIR_VALID, 0.5, False)

Found 11299 images belonging to 2 classes.
__verbose__: View > over: (<class 'keras.src.preprocessing.image.DirectoryIterator'>, 'no shape')


In [58]:
epochs = 30
add_dvc('epochs', epochs)
ratio = 1.0
add_dvc('ratio_bpe', ratio)
full_spe = 125
full_vs = 18

In [59]:
%time

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs


- training: block

In [ ]:
# hist = model.fit(train_data_gen,
#                   callbacks=[saving.getModelCheckpoint(), ModelPrint(big, 10)],
#                   epochs=epochs, steps_per_epoch=ratio * full_spe, verbose=1,
#                   validation_data=valid_data_gen, validation_steps=ratio * full_vs, validation_freq=3)

- testing

In [64]:
big = loading.flowBatches(DIR_VALID, split=0.99999, rng=False)

Found 22596 images belonging to 2 classes.
__verbose__: View > over: (<class 'keras.src.preprocessing.image.DirectoryIterator'>, 'no shape')


In [65]:
model.evaluate(big)

354/354 [==============================] - 1012s 3s/step - loss: 0.4092 - accuracy: 0.8729


[0.40917304158210754, 0.8728535771369934]

- deploy

In [66]:
model.save('model321.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [72]:
from keras.preprocessing import image
from datetime import datetime

image_path = "/content/160095.jpg"
INPUT_SIZE = (64, 64)

print("image input:")
imge = image.load_img(image_path, target_size=(64, 64))
x = image.img_to_array(imge)
print(x.shape)
x = np.expand_dims(x, axis=0) / 255
print(x.shape, type(x))
x = np.vstack([x])
print(x.shape, type(x))

print("predict:")
classes = model.predict(x, batch_size=1)
print(classes)
if classes[0]>0.5: print("This is a male")
else: print( "This is a female")

print("now:", datetime.now())

image input:
(64, 64, 3)
(1, 64, 64, 3) <class 'numpy.ndarray'>
(1, 64, 64, 3) <class 'numpy.ndarray'>
predict:
1/1 [==============================] - 0s 312ms/step
[[0.9048627]]
This is a male
now: 2024-01-29 14:01:41.932564


In [73]:
model.predict(valid_data_gen)

18/18 [==============================] - 57s 3s/step


array([[1.4368938e-08],
       [1.1668251e-05],
       [1.6284207e-01],
       ...,
       [3.1622306e-01],
       [3.5077417e-07],
       [1.6226870e-05]], dtype=float32)

## end